# Early Stage Diabetes Risk Prediction

add introduction here!!!

## Problem Statement


From dataset, trying to predict whether a patient has diabetes or not.

## The Dataset
https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("diabetes_prediction_dataset.csv")
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


There are 100,000 entries in this dataset. On first appearances, there are no null values.

## Data pre-processing

Most of the features are numerical. Only gender and smoking_history need to be handled accordingly.

In [4]:
df["gender"].unique()

array(['Female', 'Male', 'Other'], dtype=object)

Since there are 3 values for gender, I will use one-hot encoding to turn the data numerical. 

In [5]:
df = df.join(pd.get_dummies(df.gender, prefix="gender", dtype=int))

In [6]:
df["smoking_history"].value_counts()

smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64

Around 36% of entries are missing smoking_history info. This would be quite a lot of data to drop, so I will not drop them.

Instead, I will use Mode Category Imputation based on gender and age.

In [7]:
# This function takes all entries within an age range as a parameter
# It returns the most common value for 'smoking_history' that is not 'No Info'
def get_mode_smoking_history_from_group(group):
    value_counts = group.value_counts()    
    most_common = value_counts.index[0] if value_counts.index[0] != "No Info" else value_counts.index[1]    
    return most_common

# Group all entries into age_group bins of width 5 (years). 
# Replace all smoking_history values of 'No Info' with the most common value in the respective age group.
df['age_group'] = pd.cut(df['age'], bins=np.arange(0, df["age"].max(), 5), right=False)
df['smoking_history_of_age_group'] = df.groupby('age_group')['smoking_history'].transform(lambda x: get_mode_smoking_history_from_group(x))
df.loc[df['smoking_history'] == 'No Info', 'smoking_history'] = df['smoking_history_of_age_group']

# Check that there are no more counts for 'No Info'
df["smoking_history"].value_counts()

/var/folders/nl/l_thjhtd7s13pv8s4s13w7nh0000gn/T/ipykernel_31377/969091834.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['smoking_history_of_age_group'] = df.groupby('age_group')['smoking_history'].transform(lambda x: get_mode_smoking_history_from_group(x))


smoking_history
never          68123
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64